In [4]:
from pathlib import Path
from validphys.loader import _get_nnpdf_profile
from validphys.api import API
import numpy as np
import pandas as pd
from validphys.convolution import central_predictions

profile = _get_nnpdf_profile()
yaml_db = Path(profile["data_path"]) / "yamldb"

The `yaml_db` folder is a temporary thing as it contains files that look like:

```yaml
conversion_factor: 1.0
operands:
- - NMC_NC_EM_D_F2
- - NMC_NC_EM_P_F2
operation: RATIO
target_dataset: NMCPD
```

This information will eventually be part of the new commondata format of course.

The `operation` is applied to the first level of the list while the second level is just concatenated. This is necessary since `pineappl` fktables might contain one layer of concatenation which is already done for the "classic" fktables.

The `pineappl` fktables will live inside the appropiate `theory_xxx` folder `/pineappls`.

In [12]:
for i in all_res:
    print(i)

            vp      pine ratio for {'dataset': 'NMCPD_dw_ite', 'frac': 0.75}
             0         0                                                   0
data                                                                        
45    0.981587  0.981597                                           0.999990 
46    0.982138  0.982148                                           0.999990 
58    0.977590  0.977598                                           0.999992 
59    0.978155  0.978163                                           0.999992 
60    0.978707  0.978713                                           0.999994 
...        ...       ...                                                ... 
255   0.703183  0.699082                                           1.005867 
256   0.701646  0.698376                                           1.004683 
257   0.700404  0.697801                                           1.003732 
258   0.699313  0.697273                                           1.002926 

In [11]:
# Test them all
if True:
    from yaml import safe_load
    pdf = API.pdf(pdf="NNPDF40_nnlo_as_01180")
    all_res = []
    nnpdf40_runcard = safe_load(Path("/home/juacrumar/NNPDF-testing/nnpdf/n3fit/NNPDF40_with_pineappl.yml").read_text())
    for d in nnpdf40_runcard["dataset_inputs"]:
        target_ds = d["dataset"]
        cfac = []
        old_ds = API.dataset(dataset_input={"dataset": target_ds, "cfac": cfac + ["oldmode"]}, theoryid=200, use_cuts="internal")
        ds = API.dataset(dataset_input={"dataset": target_ds, "cfac": cfac}, theoryid=200, use_cuts="internal")
        new_cp = central_predictions(ds, pdf)
        cp = central_predictions(old_ds, pdf)
        all_res.append(pd.concat([new_cp, cp, new_cp/cp], axis=1, keys=["vp", "pine", f"ratio for {d}"]))

In [2]:
target_ds = "ATLAS_SINGLETOP_TCH_DIFF_7TEV_T_RAP_NORM"
cfac = []
old_ds = API.dataset(dataset_input={"dataset": target_ds, "cfac": cfac + ["oldmode"]}, theoryid=200, use_cuts="internal")
ds = API.dataset(dataset_input={"dataset": target_ds, "cfac": cfac}, theoryid=200, use_cuts="internal")

In [3]:
# Let's try to get a prediction out of it
pdf = API.pdf(pdf="NNPDF40_nnlo_as_01180")
new_cp = central_predictions(ds, pdf)
cp = central_predictions(old_ds, pdf)
pd.concat([new_cp, cp, new_cp/cp], axis=1, keys=["vp", "pine", "ratio"])


-- Reading COMMONDATA for Dataset: ATLAS_SINGLETOP_TCH_DIFF_7TEV_T_RAP_NORM
nData: 3 nSys: 17
-- COMMONDATA Files for ATLAS_SINGLETOP_TCH_DIFF_7TEV_T_RAP_NORM successfully read.

LHAPDF 6.4.0 loading /usr/share/lhapdf/LHAPDF/NNPDF40_nnlo_as_01180/NNPDF40_nnlo_as_01180_0000.dat
NNPDF40_nnlo_as_01180 PDF set, member #0, version 1


,vp,pine,ratio
,0,0,0
data,,,
0,0.631730,0.631728,1.000002
1,0.608302,0.608227,1.000123
2,0.526591,0.526411,1.000341


In [ ]:
pine_fkspec = ds.fkspecs[0]
old_fkspec = old_ds.fkspecs[0]

In [ ]:
import pineappl
pines = [pineappl.fk_table.FkTable.read(i.as_posix()) for i in pine_fkspec.fkpath]
# Inspect the pineappl prediction
res_pine = []
pp = pines[0]
lpdf = pdf.load()

for p in pines:
    res_pine.append(p.convolute_with_one(2212, lpdf.central_member.xfxQ2))
total_pine = np.concatenate(res_pine)

In [ ]:
# Let's inspect the content of the old fktables, remove the cfactor for now
from validphys.fkparser import load_fktable
old_fkspec.cfactors = False
old_fktabledata = load_fktable(old_fkspec)

In [ ]:
print(f"hadronic?: {old_fktabledata.hadronic}")
print(f"Q: {old_fktabledata.Q0}")
print(f"n: {old_fktabledata.ndata}")
print(f"xgrid shape: {old_fktabledata.xgrid.shape}")
#old_fktabledata.sigma

In [ ]:
# First read the metadata that vp `FKTableData` needs and that all subgrids share
Q0 = np.sqrt(pp.muf2())
xgrid = pp.x_grid()
# Hadronic means in practice that not all luminosity combinations are just electron X proton
hadronic = not all(-11 in i for i in pp.lumi())
# Now prepare the concatenation of grids
fktables = []
for p in pines:
    tmp = p.table().T/p.bin_normalizations()
    fktables.append(tmp.T)
fktable = np.concatenate(fktables, axis=0)
ndata = fktable.shape[0]

In [ ]:
# Now let's try to join the fktable, luminosity and xgrid into a pandas dataframe
# keeping compatibility with validphys and, hopefully, 50% of my own sanity

# Step 1), make the luminosity into a 14x14 mask for the evolution basis
eko_numbering_scheme = (22, 100, 21, 200, 203, 208, 215, 224, 235, 103, 108, 115, 124, 135)
# note that this is the same ordering that was used in fktables
flavour_map = np.zeros((14, 14), dtype=bool)
for i, j in pp.lumi():
    idx = eko_numbering_scheme.index(i)
    jdx = eko_numbering_scheme.index(j)
    flavour_map[idx,jdx] = True
    
# Step 2) prepare the indices for the dataframe
xi = np.arange(len(xgrid))
ni = np.arange(ndata)
mi = pd.MultiIndex.from_product([ni, xi, xi], names=["data", "x1", "x2"])
co = np.where(flavour_map.ravel())[0]

# Step 3) Now play with the array until we flatten it in the right way?
# The fktables for pineappl have this extra factor of x...
# The output of pineappl is (ndata, flavours, x, x)
lf = len(co)
xfktable = fktable.reshape(ndata, lf, -1)/(xgrid[:,None]*xgrid[None,:]).flatten()
fkmod = np.moveaxis(xfktable, 1, -1)
fkframe = fkmod.reshape(-1, lf)

# Uff, big
df = pd.DataFrame(fkframe, index=mi, columns=co)

from validphys.convolution import central_hadron_predictions
from validphys.coredata import FKTableData
fk = FKTableData(sigma=df, ndata=ndata,  Q0=Q0, metadata=None, hadronic=True, xgrid=xgrid)
central_hadron_predictions(fk, pdf)

In [ ]:
# Create a luminosity tensor and check that the results are correct
from validphys.pdfbases import evolution

evol_basis = (
    "photon",
    "singlet",
    "g",
    "V",
    "V3",
    "V8",
    "V15",
    "V24",
    "V35",
    "T3",
    "T8",
    "T15",
    "T24",
    "T35",
)
total_pdf = evolution.grid_values(pdf, evol_basis, xgrid, [Q0]).squeeze()[0]/xgrid
print(total_pdf.shape)
lumi = np.einsum('ij,kl->ikjl', total_pdf, total_pdf)
lumi_masked = lumi[flavour_map]
print(fktable.shape)
print(lumi_masked.shape)
res = np.einsum('ijkl,jkl->i', fktable, lumi_masked)
#pd.concat([pd.DataFrame(res), cp, pd.DataFrame(res)/cp,  ], axis=1)

In [ ]:
xfktable.reshape(48,91,-1).shape

In [ ]:
from validphys.fkparser import open_fkpath, _parse_string, _parse_header, _build_sigma
from validphys.fkparser import _parse_flavour_map, _parse_hadronic_fast_kernel
try:
    f.close()
except:
    pass
f = open_fkpath(old_fkspec.fkpath)
line_and_stream = enumerate(f, start=1)
lineno, header = next(line_and_stream)
res = {}
while True:
    marker, header_name = _parse_header(lineno, header)
    if header_name == "FastKernel":
        break
    if header_name == "FlavourMap":
        out, lineno, header = _parse_flavour_map(line_and_stream)
    else:
        out, lineno, header = _parse_string(line_and_stream)
    res[header_name] = out   

In [ ]:
res["FlavourMap"].shape

In [ ]:
i_hate_pandas = _parse_hadronic_fast_kernel(f)

In [ ]:
i_hate_pandas

In [ ]:
old_fktabledata.sigma